In [1]:
from __future__ import print_function
import sklearn
from keras.utils.np_utils import *
from keras import layers
from keras import backend as K 
from keras.layers import Input, Dropout, RNN, Dense, Concatenate, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from layers.graph import GraphConvolution
from utils import *
import labels
import difflib
import pandas as pd
import time
import os

Using TensorFlow backend.
/home/user1/.conda/envs/new_traffic6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user1/.conda/envs/new_traffic6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user1/.conda/envs/new_traffic6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user1/.c

## Data of GCN
### road feature

In [ ]:
DATASET = 'cora'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 20000
PATIENCE = 300  # early stopping patience

# Get data|
X, A1,A2,A3,y = load_data_bantian(dataset=DATASET)
# y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)
POI = {
        '26':[0,0,0,1,0,1,1,0],
        '63':[0,0,0,1,0,0,1,0],
        '62':[0,0,0,1,0,0,1,0],
        '59':[0,0,0,1,0,1,1,0],
        '60':[0,0,0,1,0,0,1,0],
        '61':[0,0,0,1,0,0,1,1],
}
data_path = '/home/user1/GCN_ori/smartTraffic/traffic_data/static_feature.npy'
static_features = np.load(data_path,allow_pickle=True)

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

def get_static_feature(road_id):
    x = static_features.tolist()[road_id]
    return list(normalization(x))+POI[road_id]

roads = [26,63,62,59,60,61]

static_features = [get_static_feature(str(road)) for road in roads]
static_features = np.array(static_features)

# Normalize X
X /= X.sum(1).reshape(-1, 1)
X.shape, static_features.shape

Loading cora dataset...
Dataset has 6 nodes, 6 edges, 42 features.


((6, 42), (6, 40))

### labels


In [7]:
def str2int(str):
    return int(str[:2])*60 + int(str[3:])

def get_encode(road_id, start_time, end_time):
    new_timing_restday = {}
    for key in list(timing_restday[road_id]):
        new_timing_restday[str2int(key)] = timing_restday[road_id][key]
    times = [key for key in new_timing_restday]
    encode = []
    start_time = str2int(start_time)
    end_time = str2int(end_time)
    for index in range(len(times)):  
        if (times[index] <= start_time and times[index + 1] > start_time):
            start_index = index
            break
    while (start_time < end_time):
        while(start_time < times[start_index + 1]):
            
            timings = new_timing_restday[times[start_index]]
            for i in range(len(timings)):
                if (road_id == '6410' and len(timings) == 4):
                    phase = phases_restday['6410-4'][i].replace(' ', '')
                elif (road_id == '6410' and len(timings) == 5):
                    phase = phases_restday['6410-5'][i].replace(' ', '')
                else:
                    phase = phases_restday[road_id][i].replace(' ', '')
                encode.extend([phase for i in range(int(timings[i]))])
            start_time += 5
            if (start_time > end_time):
              break
        start_index += 1
    return encode



ModuleNotFoundError: No module named 'Levenshtein'

### Data of LSTM

In [3]:
def getCrossFlow(df_cross, start_time, end_time):
    start = df_cross[df_cross.time==start_time].index[0]
    end = df_cross[df_cross.time==end_time].index[0]
    data_N = np.array(df_cross[start:end]['North'])
    data_W = np.array(df_cross[start:end]['West'])
    data_S = np.array(df_cross[start:end]['South'])
    data_E = np.array(df_cross[start:end]['East'])
    return np.stack([data_N, data_W, data_S, data_E],axis=1)
def getSample(roads, start_time, end_time):
    sample = []
    data_path = '/home/user1/GCN_ori/smartTraffic/traffic_data/cross/{}.csv'
    for road in roads:
        df = pd.read_csv(data_path.format(road))
        df.fillna(0,inplace = True)
        df = df.reset_index(drop=True)
        data = getCrossFlow(df,start_time, end_time)
        sample.append(data)
    return np.array(sample)
def getSample_for_road(road, start_time, end_time):
    sample = []
    data_path = '/home/user1/GCN_ori/smartTraffic/traffic_data/cross/{}.csv'
 
    df = pd.read_csv(data_path.format(road))
    df.fillna(0,inplace = True)
    df = df.reset_index(drop=True)
    data = getCrossFlow(df,start_time, end_time)
    sample.append(data)
    return np.array(sample)

# getSample(roads, '2020-10-30 00:00:00', '2020-10-30 01:00:00')
getSample_for_road(26, '2020-10-30 00:00:00', '2020-10-30 01:00:00')

array([[[21., 33., 21.,  7.],
        [19., 17., 18.,  7.],
        [19., 18., 25., 13.],
        [30., 24., 11., 10.],
        [27., 17., 13., 15.],
        [19., 18., 22., 14.],
        [17., 14., 17., 12.],
        [17., 18., 15.,  8.],
        [20.,  7., 13.,  5.],
        [15., 19., 19.,  5.],
        [15., 18.,  7.,  7.],
        [16.,  9., 14.,  4.]]])

### All data

In [2]:
days = ['2020-10-30 ', '2020-10-31 ', '2020-11-01 ', '2020-11-02 ']
hours = ['00', '01', '02','03','04','05','06', '07', '08', '09','10', '11','12', '13','14','15', '16','17', '18', '19','20', '21','22','23', '24']
start_time = '2020-10-30 08:00:00'
end_time = '2020-10-30 10:00:00'
roads = ['26','63','62','59','60','61']

#4重循环
car_flow_data = []
labels = []
for hour in tqdm(range(23)):
    for day1 in tqdm(days):
        for day2 in tqdm(days):
            for day3 in tqdm(days):
                for day4 in tqdm(days):
                    for day5 in days:
                        for day6 in days:
                            start_hour = hour
                            end_hour = hour + 1
                            if(start_hour < 10):
                                start_hour = '0'+str(start_hour)
                            else:
                                start_hour = str(start_hour)
                            if(end_hour < 10):
                                end_hour = '0'+str(end_hour)
                            else:
                                end_hour = str(end_hour)
                            start_time = start_hour +':00:00'
                            end_time = end_hour +':00:00'
                            labels.append([[similarity(road1,start_hour +':00',end_hour +':00',road2,start_hour +':00',end_hour +':00') 
                              for road2 in roads] for road1 in roads])
                            car_flow1 = getSample_for_road('26', day1 + start_time, day1 + end_time)[0]
                            car_flow2 = getSample_for_road('63', day2 + start_time, day2 + end_time)[0]
                            car_flow3 = getSample_for_road('62', day3 + start_time, day3 + end_time)[0]
                            car_flow4 = getSample_for_road('59', day4 + start_time, day4 + end_time)[0]
                            car_flow5 = getSample_for_road('60', day5 + start_time, day5 + end_time)[0]
                            car_flow6 = getSample_for_road('61', day6 + start_time, day6 + end_time)[0]
                            car_flow_data.append([car_flow1,car_flow2,car_flow3,car_flow4,car_flow5,car_flow6])
                    
# car_flow_feature.shape, static_features.shape
car_flow_data=np.array(car_flow_data)
labels = np.array(labels)
labels.shape
#6*6的矩阵
# similarity('26','07:05','08:15','63','08:05','08:15')

NameError: name 'tqdm' is not defined

((86016, 6, 6), (86016, 6, 12, 4))

## dataset


In [ ]:
labels = np.load("labels.npy")
car_flow_feature = np.load("car_flow_data.npy")
labels.shape, car_flow_feature.shape

((86016, 6, 6), (86016, 6, 12, 4))

## model

In [ ]:
# import tensorflow as tf
def slice(x,index):
    return x[:,index]
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    print("sum_squart,",sum_square)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))
input_shape = (12, 4)
output_shape = (1)
support = 1
# 第一个输入
feature_input = Input(
    shape=(static_features.shape[1:]), name='static_feature')
if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
#     print('Using local pooling filters...')
    G1 = [Input(shape=(None, None), batch_shape=(None, None),sparse=True, name="1stNeighbor")]
    G2 = [Input(shape=(None, None),  batch_shape=(None, None), sparse=True, name="2ndNeighbor")]
    G3 = [Input(shape=(None, None),  batch_shape=(None, None),sparse=True, name="3rdNeighbor")]
elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    pass
else:
    raise Exception('Invalid filter type.')
X_in = Input(shape=input_shape)
X2 = layers.LSTM(10,return_sequences=True,activation='relu')(X_in)
H = layers.Lambda(slice, arguments={'index': -1}, name='squeeze1')(X2)
H1 = GraphConvolution(16, 1, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN1_1')([H]+G1+G2+G3)
H2 = GraphConvolution(16, 2, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN2_1')([H]+G1+G2+G3)
H3 = GraphConvolution(16, 3, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN3_1')([H]+G1+G2+G3)
Y1 = GraphConvolution(y.shape[1], 1, support, name='GCN1_2')([H1]+G1+G2+G3)
Y2 = GraphConvolution(y.shape[1], 2, support, name='GCN2_2')([H2]+G1+G2+G3)
Y3 = GraphConvolution(y.shape[1], 3, support, name='GCN3_2')([H3]+G1+G2+G3)
Y = Concatenate(name="concatStaticFeature")([Y1, Y2, Y3, feature_input])
output_layer = Dense(output_shape, name="outPut")(Y)

# 第二个输入
feature_input_ = Input(
    shape=(static_features.shape[1:]), name='static_feature_')
if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
#     print('Using local pooling filters...')

    G1_ = [Input(shape=(None, None),   batch_shape=(None, None),sparse=True, name="1stNeighbor_")]
    G2_ = [Input(shape=(None, None),  batch_shape=(None, None),sparse=True, name="2ndNeighbor_")]
    G3_ = [Input(shape=(None, None),   batch_shape=(None, None),sparse=True, name="3rdNeighbor_")]
elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    pass
else:
    raise Exception('Invalid filter type.')
X_in_ = Input(shape=input_shape)
X2_ = layers.LSTM(10,return_sequences=True,activation='relu')(X_in_)

H_ = layers.Lambda(slice, arguments={'index': -1}, name='squeeze1_')(X2_)
H1_ = GraphConvolution(16, 1, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN1_1')([H_]+G1_+G2_+G3_)
H2_ = GraphConvolution(16, 2, support, activation='relu', kernel_regularizer=l2(
    5e-4), name='GCN2_1_')([H_]+G1_+G2_+G3_)
H3_ = GraphConvolution(16, 3, support, activation='relu', kernel_regularizer=l2(
    5e-4), name='GCN3_1_')([H_]+G1_+G2_+G3_)
Y1_ = GraphConvolution(y.shape[1], 1, support,
                       name='GCN1_2_')([H_]+G1_+G2_+G3_)
Y2_ = GraphConvolution(y.shape[1], 2, support,
                       name='GCN2_2_')([H_]+G1_+G2_+G3_)
Y3_ = GraphConvolution(y.shape[1], 3, support,
                       name='GCN3_2_')([H_]+G1_+G2_+G3_)
Y_ = Concatenate(name="concatStaticFeature_")([Y1_, Y2_, Y3_, feature_input_])
output_layer_ = Dense(output_shape, name="outPut_")(Y_)
distance = Lambda(euclidean_distance, output_shape=(6,), name="distance")([output_layer, output_layer_])

all_input = [X_in]+G1+G2+G3+[feature_input] + \
    [X_in_]+G1_+G2_+G3_+[feature_input_]
model = Model(inputs=all_input, outputs=distance)

def r_square(y_true, y_pred):
    SSR = K.mean(K.square(y_pred-K.mean(y_true)),axis=-1)
    SST = K.mean(K.square(y_true-K.mean(y_true)),axis=-1)
    return SSR/SST
# from sklearn.metrics import explained_variance_score
# def evs(y_true, y_pred):
#     return explained_variance_score(np.array(y_true), np.array(y_pred))
# from tensorflow.python.keras.metrics import Metric
from keras import metrics
model.compile(loss='mse', optimizer=Adam(lr=0.001),metrics = [metrics.mean_squared_error,metrics.mean_absolute_error,metrics.binary_accuracy,r_square])
model.summary()

sum_squart, Tensor("distance/Sum:0", shape=(?, 1), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 4)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 10)       600         input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 12, 4)        0                                            
__________________________________________________________________________________________________
squeeze1 (Lambda)               (None, 10)           0           lstm_1[0][0]                     
___________________________________________

In [ ]:
A1 = preprocess_adj(A1, SYM_NORM)
A2 = preprocess_adj(A2, SYM_NORM)
A3 = preprocess_adj(A3, SYM_NORM)
# y_train = np.array([1 for i in range(6)])

labels.reshape([len(labels),6,-1]).shape

(86016, 6, 6)

In [19]:
model.metrics_names
np.array([2,2])/2

array([1., 1.])

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
l = len(car_flow_feature)
l=12000
ll = int(l*0.8)
ans = np.array([0,0,0,0,0])

for i in range(ll):
    graph = [car_flow_feature[i], A1,A2,A3,static_features]
    model.fit(graph+graph,labels[i], 
                batch_size=A1.shape[0] , epochs=10, shuffle=False, verbose=0)
cnt = 0
for i in range(ll, l):
    cnt = cnt + 1
    graph = [car_flow_feature[i], A1,A2,A3,static_features]
    res = model.evaluate(graph+graph,labels[i])
    ans = np.sum([ans,res], axis = 0)
print(ans/cnt)

6/6 [==============================] - 0s 4ms/step
[0.0378957  0.03693414 0.15791942 0.16666667 1.28761274]


In [21]:
!nvidia-smi

Sun May 22 14:38:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:3D:00.0 Off |                  N/A |
| 41%   27C    P8    15W / 280W |   4292MiB / 24219MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:3E:00.0 Off |                  N/A |
| 41%   32C    P8    14W / 280W |     12MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## train

In [15]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A1.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A1.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 1.5191 train_acc= 1.0000 val_loss= 1.5191 val_acc= 1.0000 time= 9.6271
Epoch: 0002 train_loss= 1.8975 train_acc= 1.0000 val_loss= 1.8975 val_acc= 1.0000 time= 0.0094
Epoch: 0003 train_loss= 2.2486 train_acc= 1.0000 val_loss= 2.2486 val_acc= 1.0000 time= 0.0152
Epoch: 0004 train_loss= 2.5003 train_acc= 1.0000 val_loss= 2.5003 val_acc= 1.0000 time= 0.0148
Epoch: 0005 train_loss= 2.5939 train_acc= 1.0000 val_loss= 2.5939 val_acc= 1.0000 time= 0.0142
Epoch: 0006 train_loss= 2.5311 train_acc= 1.0000 val_loss= 2.5311 val_acc= 1.0000 time= 0.0127
Epoch: 0007 train_loss= 2.3661 train_acc= 1.0000 val_loss= 2.3661 val_acc= 1.0000 time= 0.0176
Epoch: 0008 train_loss= 2.1610 train_acc= 1.0000 val_loss= 2.1610 val_acc= 1.0000 time= 0.0191
Epoch: 0009 train_loss= 1.9552 train_acc= 1.0000 val_loss= 1.9552 val_acc= 1.0000 time= 0.0220
Epoch: 0010 train_loss= 1.7667 train_acc= 1.0000 val_loss= 1.7667 val_acc= 1.0000 time= 0.0186
Epoch: 0011 train_loss= 1.6182 train_acc= 1.0000 v

KeyboardInterrupt: 

## backup

In [21]:
def slice(x,index):
    return x[:,:,index]

def create_base_network(input_shape):
    feature_input = Input(shape=(static_features.shape[1],), sparse=True, name = 'static_feature')
    if FILTER == 'localpool':
        """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
        print('Using local pooling filters...')
        A1_ = preprocess_adj(A1, SYM_NORM)
        A2_ = preprocess_adj(A2, SYM_NORM)
        A3_ = preprocess_adj(A3, SYM_NORM)
        support = 1
        graph = [X, A1,A2,A3]
        G1 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "1stNeighbor")]
        G2 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "2ndNeighbor")]
        G3 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "3rdNeighbor")]

    elif FILTER == 'chebyshev':
        """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
        print('Using Chebyshev polynomial basis filters...')
        L = normalized_laplacian(A, SYM_NORM)
        L_scaled = rescale_laplacian(L)
        T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
        support = MAX_DEGREE + 1
        graph = [X]+T_k
        G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(support)]

    else:
        raise Exception('Invalid filter type.')
    X_in = Input(shape=input_shape)
    X1 = layers.LSTM(1, return_sequences = True, name = 'LSTM')(X_in)
    H = layers.Lambda(slice,arguments={'index':-1}, name = 'squeeze')(X1)
    H1 = GraphConvolution(16, 1,support, activation='relu', kernel_regularizer=l2(5e-4), name = 'GCN1_1')([H]+G1+G2+G3)
    H2 = GraphConvolution(16, 2,support, activation='relu', kernel_regularizer=l2(5e-4), name = 'GCN2_1')([H]+G1+G2+G3)
    H3 = GraphConvolution(16, 3,support, activation='relu', kernel_regularizer=l2(5e-4), name = 'GCN3_1')([H]+G1+G2+G3)
    Y1 = GraphConvolution(y.shape[1],1 ,support, name = 'GCN1_2')([H1]+G1+G2+G3)
    Y2 = GraphConvolution(y.shape[1],2, support, name = 'GCN2_2')([H2]+G1+G2+G3)
    Y3 = GraphConvolution(y.shape[1],3, support, name = 'GCN3_2')([H3]+G1+G2+G3)
    Y = Concatenate(name="concatStaticFeature")([Y1,Y2,Y3,feature_input])
    Y = Dense(y.shape[1], name="outPut")(Y)
    model = Model(inputs=[X_in]+G1+G2+G3+[feature_input], outputs=Y)
    return model

# Siamese RNN
# input_shape = (2015,1)

base_network = create_base_network((car_flow_feature.shape[1:]))

feature_input = Input(shape=(static_features.shape[1],), sparse=True, name = 'static_feature')
X_in = Input(shape=input_shape)
G1 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "1stNeighbor")]
G2 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "2ndNeighbor")]
G3 = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name = "3rdNeighbor")]

input_a = [X_in]+G1+G2+G3+[feature_input]
# print(base_network.summary())
# input_a = Input(shape=input_shape)
processed_a = base_network(input_a)
# processed_b = base_network(input_b)

# distance = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([processed_a, processed_b])
# model = Model([input_a, input_b], outputs=processed_a)

# # model = Model(inputs=[X_in]+G1+G2+G3, outputs=Y)

# base_network.compile(loss='mse', optimizer=Adam(lr=0.001))


Using local pooling filters...


Exception: Layers should have equal number of output tensors and output masks. Layer GCN3_1 has 1 output tensors and 4 output masks.

In [ ]:
def slice(x,index):
    return x[:,index]
input_shape = (17, 12, 4)
output_shape = (1)
# 第一个输入
feature_input = Input(
    shape=(static_features.shape), name='static_feature')
feature_input_low = layers.Lambda(slice, arguments={'index': -1}, name='squeeze0')(feature_input)

support = 1

if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
#     print('Using local pooling filters...')

    G1 = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="1stNeighbor")]
    G2 = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="2ndNeighbor")]
    G3 = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="3rdNeighbor")]
elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    pass
else:
    raise Exception('Invalid filter type.')
X_in = Input(shape=input_shape)
X1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)(X_in)
X2 = layers.MaxPooling2D((2, 2))(X1)

H = layers.Lambda(slice, arguments={'index': -1}, name='squeeze1')(X2)
H = layers.Lambda(slice, arguments={'index': -1}, name='squeeze2')(H)
# H = X3
H1 = GraphConvolution(16, 1, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN1_1')([H]+G1+G2+G3)
H2 = GraphConvolution(16, 2, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN2_1')([H]+G1+G2+G3)
H3 = GraphConvolution(16, 3, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN3_1')([H]+G1+G2+G3)
Y1 = GraphConvolution(y.shape[1], 1, support, name='GCN1_2')([H1]+G1+G2+G3)
Y2 = GraphConvolution(y.shape[1], 2, support, name='GCN2_2')([H2]+G1+G2+G3)
Y3 = GraphConvolution(y.shape[1], 3, support, name='GCN3_2')([H3]+G1+G2+G3)
Y = Concatenate(name="concatStaticFeature")([Y1, Y2, Y3, feature_input_low])
output_layer = Dense(output_shape, name="outPut")(Y)

# 第二个输入
feature_input_ = Input(
    shape=(static_features.shape), name='static_feature_')
feature_input_low_ = layers.Lambda(slice, arguments={'index': -1}, name='squeeze0_')(feature_input_)

if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
#     print('Using local pooling filters...')

    G1_ = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="1stNeighbor_")]
    G2_ = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="2ndNeighbor_")]
    G3_ = [Input(shape=(None, None), batch_shape=(
        None, None), sparse=True, name="3rdNeighbor_")]
elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    pass
else:
    raise Exception('Invalid filter type.')
X_in_ = Input(shape=input_shape)
X1_ = layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)(X_in_)
X2_ = layers.MaxPooling2D((2, 2))(X1_)

H_ = layers.Lambda(slice, arguments={'index': -1}, name='squeeze1_')(X2_)
H_ = layers.Lambda(slice, arguments={'index': -1}, name='squeeze2_')(H_)
# H = X3
H1_ = GraphConvolution(16, 1, support, activation='relu',
                      kernel_regularizer=l2(5e-4), name='GCN1_1')([H_]+G1_+G2_+G3_)
H2_ = GraphConvolution(16, 2, support, activation='relu', kernel_regularizer=l2(
    5e-4), name='GCN2_1_')([H_]+G1_+G2_+G3_)
H3_ = GraphConvolution(16, 3, support, activation='relu', kernel_regularizer=l2(
    5e-4), name='GCN3_1_')([H_]+G1_+G2_+G3_)
Y1_ = GraphConvolution(y.shape[1], 1, support,
                       name='GCN1_2_')([H_]+G1_+G2_+G3_)
Y2_ = GraphConvolution(y.shape[1], 2, support,
                       name='GCN2_2_')([H_]+G1_+G2_+G3_)
Y3_ = GraphConvolution(y.shape[1], 3, support,
                       name='GCN3_2_')([H_]+G1_+G2_+G3_)
Y_ = Concatenate(name="concatStaticFeature_")([Y1_, Y2_, Y3_, feature_input_low_])
output_layer_ = Dense(output_shape, name="outPut_")(Y_)


distance = Lambda(euclidean_distance, output_shape=(1,), name="distance")([output_layer, output_layer_])
final_output = Dense(units = 1,name="final_outPut")(distance)
all_input = [X_in]+G1+G2+G3+[feature_input] + \
    [X_in_]+G1_+G2_+G3_+[feature_input_]
model = Model(inputs=all_input, outputs=final_output)
model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.summary()